In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_theme()

/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#import and clean datasets
advanced = pd.read_csv("advanced.csv", header = 1, index_col=1).iloc[1:].dropna(axis=1, how='all')
per_game = pd.read_csv('per_game.csv', index_col=1)
standings = pd.read_csv('standings.csv', header = 1, index_col=1)

#remove astericks in team names
per_game.index = per_game.index.str.replace(r'\*', '', regex=True)
advanced.index = advanced.index.str.replace(r'\*', '', regex=True)

#merge datasets
##TODO: merge datasets for all three
##TODO: rename columns in standings dataset. Add conference/division labels to colums
merged = pd.concat([standings, per_game, advanced], axis=1, join='outer')

In [3]:
try:
    import sklearn
    print(sklearn.__version__)
    if (sklearn.__version__ < '1.3.0'): #update for latest scikit-learn version
        !pip install -U --user scikit-learn
except Exception as e:
    !pip install scikit-learn

1.4.1.post1


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import PolynomialFeatures

In [5]:
data = merged

# Step 2: Prepare the data
X = data[['3P%']]  # Predictor variable (reshape for a single feature)
y = data['PTS']  # Response variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
new_3P = np.array([[0.35]])  # Example 3P% value
predicted_pts = model.predict(new_3P)
print(f"Predicted PTS for 3P% = 0.35: {predicted_pts[0]}")

RMSE: 2.4531407664302716
Coefficient: 72.91551246537384
Intercept: 88.91222299168982
Predicted PTS for 3P% = 0.35: 114.43265235457065


/Users/edmondniu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [6]:
merged = merged.drop(merged.index[-1])
merged[['Win', 'Loss']] = merged['Overall'].str.split('-', expand=True).astype(int)
merged['WinLossPercentage'] = merged['Win']/merged['Loss']
merged.head(5)

,Rk,Overall,Home,Road,E,W,A,C,SE,NW,...,eFG%.1,TOV%.1,DRB%,FT/FGA.1,Arena,Attend.,Attend./G,Win,Loss,WinLossPercentage
Team,,,,,,,,,,,,,,,,,,,,,
Milwaukee Bucks,1.0,58-24,32-9,26-15,35-17,23-7,6-Dec,5-Nov,6-Dec,1-Sep,...,0.520,10.4,77.8,0.175,Fiserv Forum,"718,786","17,531",58,24,2.416667
Boston Celtics,2.0,57-25,32-9,25-16,34-18,23-7,5-Nov,7-Nov,6-Dec,3-Jul,...,0.528,11.3,78.5,0.180,TD Garden,"766,240","18,689",57,25,2.280000
Philadelphia 76ers,3.0,54-28,29-12,25-16,34-18,20-10,6-Oct,13-5,7-Nov,3-Jul,...,0.541,13.0,77.2,0.217,Wells Fargo Center,"839,261","20,470",54,28,1.928571
Denver Nuggets,4.0,53-29,34-7,19-22,19-11,34-18,6-Apr,3-Jul,2-Aug,6-Oct,...,0.543,12.2,76.4,0.201,Ball Arena,"788,635","19,235",53,29,1.827586
Cleveland Cavaliers,5.0,51-31,31-10,20-21,34-18,17-13,10-Aug,13-3,13-5,6-Apr,...,0.535,14.4,76.3,0.210,Rocket Mortgage Fieldhouse,"777,280","18,958",51,31,1.645161


### Top 3 Rsquared ###

In [29]:
data = merged

# Step 2: Prepare the data
X = data[['DRtg', 'SRS', 'NRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.25487131264338586
R-squared: 0.27120484705090875
Coefficient: -0.011625038727105897
Intercept: 2.49184856254315


### Top 3 with k fold cross validation ###

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
import numpy as np

# Load your data - replace this with your actual data loading step
data = merged

# Define predictor and response variables
X = data[['DRtg', 'SRS', 'NRtg']]
y = data['WinLossPercentage']

# Apply polynomial transformation if needed
poly = PolynomialFeatures(1, include_bias=False)
poly_X = poly.fit_transform(X)

model = LinearRegression()

# Perform 5-fold cross-validation
scores = cross_val_score(model, poly_X, y, cv=5, scoring='r2')  # Use R^2 as the scoring method

print("Cross-validated R^2 scores:", scores)
print("Average R^2 score:", np.mean(scores))
print("Standard deviation of R^2 scores:", np.std(scores))

Cross-validated R^2 scores: [ -4.17217444  -0.93802222 -13.10900449 -10.28579038 -18.08748786]
Average R^2 score: -9.31849587911473
Standard deviation of R^2 scores: 6.146800755880224
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=10, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END max_depth=20, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   0.0s
[CV] END m

### Defensive Rating only R-squared: 0.41 ###

In [8]:
data = merged

# Step 2: Prepare the data
X = data[['DRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
off_def = np.array([[111.5]])  # Example 3P% value
predicted_pts = model.predict(off_def)
print(f"Predicted Win/Loss Percent for defense: 111.5: {predicted_pts[0]}")

RMSE: 0.2821292854707832
R-squared: 0.10698263870158542
Coefficient: -0.16278201926027802
Intercept: 19.852385616350613
Predicted Win/Loss Percent for defense: 111.5: 1.7021904688296132


/Users/edmondniu/anaconda3/lib/python3.11/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


### Offensive Rating only: R-squared: -0.144 ###

In [9]:
data = merged

# Step 2: Prepare the data
X = data[['ORtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0]])  # Example 3P% value
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offensive: 118.0: {predicted_pts[0]}")

RMSE: 0.20860164095634995
R-squared: 0.511798617221255
Coefficient: 0.14655654830043102
Intercept: -15.67922281272551


### Offensive and Defensive Rating R-squared: 0.44 ###

In [10]:
data = merged

# Step 2: Prepare the data
X = data[['ORtg', 'DRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
predicted_pts = model.predict(off_def)
print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.2510575738557826
R-squared: 0.2928521594512432
Coefficient: 0.11034827868882317
Intercept: 3.4605727693587944
Predicted Win/Loss Percent for offense: 118, defense: 111.5: 1.9339708876973818


### SRS R-squared: 0.49 ###

In [11]:
data = merged

# Step 2: Prepare the data
X = data[['SRS']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.2449217269255313
R-squared: 0.32699515591434003
Coefficient: 0.12413066048660458
Intercept: 1.1528553447647156


### NRtg: R-squared: 0.45 ###

In [12]:
data = merged

# Step 2: Prepare the data
X = data[['NRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.2405471518048955
R-squared: 0.35082168807091296
Coefficient: 0.1202006724040661
Intercept: 1.1489830489501536


### TSpercent: R-squared: -0.178 ###

In [13]:
data = merged

# Step 2: Prepare the data
X = data[['TS%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3164812595486796
R-squared: -0.12372375292977877
Coefficient: 20.749568850711626
Intercept: -10.938905357644419


### FTr R-squared: -0.015 ####

In [14]:
data = merged

# Step 2: Prepare the data
X = data[['FTr']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3645827133478031
R-squared: -0.49126803792883833
Coefficient: -3.777457640710367
Intercept: 2.1765814826392593


### 3PAr R-squared: 0.08 ###

In [15]:
data = merged

# Step 2: Prepare the data
X = data[['3PAr']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3047520420694139
R-squared: -0.04197383384490405
Coefficient: 4.23312915201547
Intercept: -0.5028430806233994


### Field Goal percentage R-squared: -0.22 ###

In [16]:
data = merged

# Step 2: Prepare the data
X = data[['FG%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.34508113840616633
R-squared: -0.3359986165436817
Coefficient: 22.85165153171467
Intercept: -9.71713708456126


### 3P%: R-squared: -0.010 ###

In [17]:
data = merged

# Step 2: Prepare the data
X = data[['3P%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.2668063164239042
R-squared: 0.20135132014027013
Coefficient: 21.29356956788881
Intercept: -6.5272922011291366


### 2P%: R-squared: -0.18 ###

In [18]:
data = merged

# Step 2: Prepare the data
X = data[['2P%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.4050138079299206
R-squared: -0.840361856854843
Coefficient: 18.277503747590526
Intercept: -8.906656322307835


### FT% R-squared: -0.213 ###

In [19]:
data = merged

# Step 2: Prepare the data
X = data[['FT%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(poly_X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3681633730934314
R-squared: -0.5207041281726315
Coefficient: 1.213095583383542
Intercept: 0.21722833510299577


### 3P, 2P, FT % R-squared: -0.07 ###

In [20]:
data = merged

# Step 2: Prepare the data
X = data[['FT%', '3P%', '2P%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
#poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Create the model
model = LinearRegression()

# Step 4: Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, predictions)  # This is how you calculate R-squared
print(f"RMSE: {rmse}")
print(f"R-squared: {r2}")  # Print the R-squared value


# Coefficients and Intercept
print(f"Coefficient: {model.coef_[0]}")
print(f"Intercept: {model.intercept_}")

# Predicting with a new 3P% value
#off_def = np.array([[118.0,111.5]])  # Example 3P% value
#off_def = PolynomialFeatures(3, include_bias=False).fit_transform(off_def)
#predicted_pts = model.predict(off_def)
#print(f"Predicted Win/Loss Percent for offense: 118, defense: 111.5: {predicted_pts[0]}")

RMSE: 0.3091046511575338
R-squared: -0.07195028554689098
Coefficient: -0.36134271389535716
Intercept: -9.891136937250721


### LASSO CV MODEL:  FT, 3P, 2P Lasso R-squared: 0.144 ###

In [21]:
from sklearn.linear_model import LassoCV

data = merged

# Step 2: Prepare the data
X = data[['FT%', '3P%', '2P%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
#poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the LassoCV model
lasso = LassoCV(cv=5, random_state=42)

# Train the model
lasso.fit(X_train, y_train)

# Evaluate the model
lasso_predictions = lasso.predict(X_test)
lasso_mse = mean_squared_error(y_test, lasso_predictions)
lasso_rmse = np.sqrt(lasso_mse)
lasso_r2 = r2_score(y_test, lasso_predictions)

print(f"Lasso RMSE: {lasso_rmse}")
print(f"Lasso R-squared: {lasso_r2}")

Lasso RMSE: 0.27607442453200903
Lasso R-squared: 0.14490195132058903


### Random Forest: FT, 3P, 2P RMSE: 0.48330224154176904 ###

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = merged

# Step 2: Prepare the data
X = data[['FT%', '3P%', '2P%']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
#poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Sample data preparation
# X and y should already be defined as your features and target variable respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest regressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
predictions = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

# Get feature importance
importances = rf.feature_importances_
print(f"Feature Importances: {importances}")

RMSE: 0.3473404880759311
Feature Importances: [0.2971464  0.33484028 0.36801332]


### Random Forest: SRS, NRtg, RMSE: 0.323 ###

In [34]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

data = merged

# Step 2: Prepare the data
X = data[['SRS', 'NRtg']]  # Predictor variable (reshape for a single feature)
y = data['WinLossPercentage']  # Response variable
#poly_X = PolynomialFeatures(1, include_bias=False).fit_transform(X)

# Sample data preparation
# X and y should already be defined as your features and target variable respectively
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a random forest regressor model
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Make predictions
predictions = rf.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"RMSE: {rmse}")

r2 = r2_score(y_test, predictions)
print(f"R squared: {r2}")

# Get feature importance
importances = rf.feature_importances_
print(f"Feature Importances: {importances}")

RMSE: 0.1196272793237576
R squared: 0.8394449331696743
Feature Importances: [0.52088464 0.47911536]


### Random Forest SRS NRTG with 5-fold cross validation ###
##### Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
##### RMSE: 0.10574056628191948

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

data = merged  # assuming 'merged' is your DataFrame

# Prepare the data
X = data[['SRS', 'NRtg']]  # Predictor variables
y = data['WinLossPercentage']  # Response variable

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model
rf = RandomForestRegressor(random_state=42)

# Set up the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # minimum number of samples required to be at a leaf node
}

# Create GridSearchCV object
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='r2', verbose=2, n_jobs=-1)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_rf = grid_search.best_estimator_

# Make predictions with the best model
predictions = best_rf.predict(X_test)

# Evaluate the best model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"RMSE: {rmse}")

r2 = r2_score(y_test, predictions)
print(f"R squared: {r2}")

# Get feature importance
importances = best_rf.feature_importances_
print(f"Feature Importances: {importances}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits


/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/Users/edmondniu/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWar

Best Parameters: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
RMSE: 0.10574056628191948
R squared: 0.8745568855290999
Feature Importances: [0.57375255 0.42624745]
